In [19]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
from supabase import create_client
from dotenv import load_dotenv

load_dotenv()

True

In [20]:
url = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_KEY")
supabase = create_client(url, key)

In [21]:
# Fetch data from the view
response = supabase.table("sector_etf_flow_daily").select("*").execute()
data = response.data
df = pd.DataFrame(data)

# Check columns to identify the date column
date_col = 'date' if 'date' in df.columns else 'asof_date'
if date_col in df.columns:
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(by=date_col)
    print(f"Date column used: {date_col}")
else:
    print("Date column not found.")
    print(df.columns)

Date column used: asof_date


In [22]:
if not df.empty and date_col in df.columns:
    # Set plot style
    sns.set_theme(style="white")

    # Get unique tickers
    tickers = df['ticker'].unique()

    # Create subplots: 1 column, N rows
    fig, axes = plt.subplots(len(tickers), 1, figsize=(12, 6 * len(tickers)))

    # Handle single ticker case
    if len(tickers) == 1:
        axes = [axes]

    for i, ticker in enumerate(tickers):
        ax1 = axes[i]
        ticker_data = df[df['ticker'] == ticker]
        
        # --- Plot NAV on Left Axis (Line) ---
        color_nav = 'tab:blue'
        ax1.plot(ticker_data[date_col], ticker_data['nav'], 
                 color=color_nav, marker='o', label='NAV', linewidth=2)
        
        ax1.set_ylabel('NAV ($)', color=color_nav, fontsize=12)
        ax1.tick_params(axis='y', labelcolor=color_nav)
        ax1.set_title(f'{ticker} - NAV & Flow %', fontsize=16)
        ax1.set_xlabel('Date')
        ax1.grid(True, linestyle="--", alpha=0.5)
        
        # Format x-axis dates
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        ax1.xaxis.set_major_locator(mdates.AutoDateLocator())
        
        # --- Plot Flow % on Right Axis (Line) ---
        ax2 = ax1.twinx()
        color_flow = 'tab:orange'
        
        # Changed to plot (line) as requested
        ax2.plot(ticker_data[date_col], ticker_data['flow_pct'], 
                 color=color_flow, marker='x', linestyle='--', label='Flow %', linewidth=2)
        
        ax2.set_ylabel('Flow %', color=color_flow, fontsize=12)
        ax2.tick_params(axis='y', labelcolor=color_flow)
        ax2.yaxis.set_major_formatter(mtick.PercentFormatter(1.0)) 
        
        # Add zero line for flow
        ax2.axhline(0, color='gray', linestyle=':', linewidth=1)

    plt.tight_layout()
    plt.show()